<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-dependencies" data-toc-modified-id="Load-dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load dependencies</a></span></li><li><span><a href="#Read-csv-files-to-DFs" data-toc-modified-id="Read-csv-files-to-DFs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read csv files to DFs</a></span></li><li><span><a href="#Fill-missing-values" data-toc-modified-id="Fill-missing-values-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fill missing values</a></span></li><li><span><a href="#Add-date-part" data-toc-modified-id="Add-date-part-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add date part</a></span></li><li><span><a href="#Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc." data-toc-modified-id="Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc.-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Add extra columns (purchased on weekend, monthend, month_diff etc.</a></span></li><li><span><a href="#Mark-as-categorical-variables" data-toc-modified-id="Mark-as-categorical-variables-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Mark as categorical variables</a></span></li><li><span><a href="#One-hot-encoding-categories-(not-working)" data-toc-modified-id="One-hot-encoding-categories-(not-working)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>One hot encoding categories (not working)</a></span></li><li><span><a href="#Aggregate-by-card_id" data-toc-modified-id="Aggregate-by-card_id-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Aggregate by card_id</a></span></li><li><span><a href="#Add-exta-interpreted-columns-on-aggregates" data-toc-modified-id="Add-exta-interpreted-columns-on-aggregates-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Add exta interpreted columns on aggregates</a></span></li><li><span><a href="#Load-test-&amp;-train-DFs" data-toc-modified-id="Load-test-&amp;-train-DFs-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Load test &amp; train DFs</a></span></li><li><span><a href="#Add-date-part-to-test-&amp;-train-dfs" data-toc-modified-id="Add-date-part-to-test-&amp;-train-dfs-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Add date part to test &amp; train dfs</a></span></li><li><span><a href="#Merge-train-&amp;-test-with-new-&amp;-old-transactions-history" data-toc-modified-id="Merge-train-&amp;-test-with-new-&amp;-old-transactions-history-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Merge train &amp; test with new &amp; old transactions history</a></span></li><li><span><a href="#Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc" data-toc-modified-id="Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Add extra columns like age, total transactions, installments, purchase amount, first buy etc</a></span></li><li><span><a href="#Mark-the-outliers" data-toc-modified-id="Mark-the-outliers-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Mark the outliers</a></span></li><li><span><a href="#Target-encode-the-outliers" data-toc-modified-id="Target-encode-the-outliers-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Target encode the outliers</a></span></li></ul></div>

1. Load dependencies
2. Read csv files to dataframes
3. Fill missing values
4. Add date part 
5. Add extra columns (purchased on weekend, monthend, month_diff etc.
6. Aggregate by card_id
7. Aggregate by categories
8. Mark categorical columns
9. Add exta interpreted columns on aggregates
10. Load test & train csvs to dfs
11. Add date part to test & train dfs
12. Merge train & test with new & old transactions history
13. Add extra columns like age, total transactions, installments, purchase amount, first buy etc
14. Mark the outliers
15. Target encode the outliers 
16. Save to feather

### Load dependencies

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather

### Read csv files to DFs

In [3]:
PATH = '../data/elo/'

In [4]:
files = ['historical_transactions', 'new_merchant_transactions']

In [5]:
hist_trans, new_hist_trans = [pd.read_csv(f'{PATH}{c}.csv') for c in files]

### Fill missing values

In [6]:
def fill_nas_for_transactions_df(df):
    # Fill nas for category_3 with mode
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    return df

In [7]:
dfs = [hist_trans, new_hist_trans]

In [8]:
hist_trans, new_hist_trans = [fill_nas_for_transactions_df(df) for df in dfs]

### Add date part

In [9]:
add_datepart(hist_trans, 'purchase_date', drop=False)
add_datepart(new_hist_trans, 'purchase_date', drop=False)

### Add extra columns (purchased on weekend, monthend, month_diff etc.

In [13]:
def add_extra_cols(df):
    df['purchased_on_weekend'] = (df.purchase_Dayofweek >=5).astype(int)
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    return df

In [14]:
hist_trans, new_hist_trans = [add_extra_cols(df) for df in dfs]

In [15]:
hist_trans.shape, new_hist_trans.shape

((29112361, 29), (1963031, 29))

### Mark as categorical variables

In [13]:
def mark_categorical_cols_for_transactions(df):
    categorical_cols = ['authorized_flag', 'card_id', 'city_id',
       'merchant_category_id', 'merchant_id',
       'state_id', 'subsector_id', 'purchase_Year', 'purchase_Month']
    for c in categorical_cols:
        df[c] = df[c].astype('category').cat.as_ordered()
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

In [14]:
hist_trans_c, new_hist_trans_c = [mark_categorical_cols_for_transactions(df) for df in dfs]

In [15]:
hist_trans.shape, new_hist_trans.shape

((29112361, 29), (1963031, 29))

In [16]:
hist_trans.to_feather('hist_trans')
new_hist_trans.to_feather('new_hist_trans')

In [17]:
hist_trans = feather.read_dataframe('hist_trans')

In [18]:
new_hist_trans = feather.read_dataframe('new_hist_trans')

### One hot encoding categories (not working)

In [19]:
DataFrameSummary(new_hist_trans_c).summary()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,purchase_Dayofyear,purchase_Is_month_end,purchase_Is_month_start,purchase_Is_quarter_end,purchase_Is_quarter_start,purchase_Is_year_end,purchase_Is_year_start,purchase_Elapsed,purchased_on_weekend,month_diff
count,1.96303e+06,NaN,NaN,1.96303e+06,1.96303e+06,NaN,NaN,NaN,1.96303e+06,1.96303e+06,...,1.96303e+06,NaN,NaN,NaN,NaN,NaN,NaN,1.96303e+06,1.96303e+06,1.96303e+06
mean,1,NaN,NaN,0.0321421,0.682964,NaN,NaN,NaN,1.47652,-0.550969,...,109.884,NaN,NaN,NaN,NaN,NaN,NaN,1.51935e+09,0.307438,11.772
std,0,NaN,NaN,0.176378,1.58407,NaN,NaN,NaN,0.499448,0.694004,...,75.1097,NaN,NaN,NaN,NaN,NaN,NaN,6.6753e+06,0.461433,2.55921
min,1,NaN,NaN,0,-1,NaN,NaN,NaN,1,-0.746893,...,1,NaN,NaN,NaN,NaN,NaN,NaN,1.48834e+09,0,10
25%,1,NaN,NaN,0,0,NaN,NaN,NaN,1,-0.716629,...,71,NaN,NaN,NaN,NaN,NaN,NaN,1.51967e+09,0,10
50%,1,NaN,NaN,0,1,NaN,NaN,NaN,1,-0.674841,...,90,NaN,NaN,NaN,NaN,NaN,NaN,1.52146e+09,0,11
75%,1,NaN,NaN,0,1,NaN,NaN,NaN,2,-0.581616,...,112,NaN,NaN,NaN,NaN,NaN,NaN,1.52322e+09,1,12
max,1,NaN,NaN,1,999,NaN,NaN,NaN,2,263.157,...,365,NaN,NaN,NaN,NaN,NaN,NaN,1.52513e+09,1,23
counts,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,...,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031
uniques,1,290001,308,2,15,3,314,226129,2,75190,...,365,2,2,2,2,2,2,1667025,2,14


In [20]:
def one_hot_encode_dfs(df):
    return pd.get_dummies(df, columns=['category_2', 'category_3'])

In [21]:
hist_trans_c.shape, new_hist_trans_c.shape

((29112361, 29), (1963031, 29))

In [22]:
hist_trans_c_e, new_hist_trans_c_e = [one_hot_encode_dfs(df) for df in dfs]

In [23]:
hist_trans_c_e.shape, new_hist_trans_c_e.shape

((29112361, 35), (1963031, 35))

In [24]:
new_hist_trans_c_e.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount',
       'purchase_date', 'state_id', 'subsector_id', 'purchase_Year',
       'purchase_Month', 'purchase_Week', 'purchase_Day', 'purchase_Dayofweek',
       'purchase_Dayofyear', 'purchase_Is_month_end',
       'purchase_Is_month_start', 'purchase_Is_quarter_end',
       'purchase_Is_quarter_start', 'purchase_Is_year_end',
       'purchase_Is_year_start', 'purchase_Elapsed', 'purchased_on_weekend',
       'month_diff', 'category_2_1.0', 'category_2_2.0', 'category_2_3.0',
       'category_2_4.0', 'category_2_5.0', 'category_3_A', 'category_3_B',
       'category_3_C'],
      dtype='object')

In [25]:
## Not working
def aggregate_by_category(df):
    features = ['category_2_1.0', 'category_2_2.0', 'category_2_3.0',
       'category_2_4.0', 'category_2_5.0', 'category_3_A', 'category_3_B',
       'category_3_C']
    cat_agg = {"purchase_amount": ["sum", "mean"], "installments": ["sum", "mean"]}
    other_df = pd.DataFrame()
    for fe in features:
        g = df.groupby(['card_id', fe]).agg(cat_agg)
        g.columns = ['_'+fe+'_'.join(c).strip() for c in g.columns.values]
        g.reset_index(inplace=True)
        if other_df.empty:
            other_df = g
        else:
            other_df = pd.merge(other_df, g, on='card_id', how='left')
    return other_df

### Aggregate by card_id

In [16]:
hist_trans.head().T

,0,1,2,3,4
authorized_flag,1,1,1,1,1
card_id,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc
city_id,88,88,88,88,88
category_1,0,0,0,0,0
installments,0,0,0,0,0
category_3,A,A,A,A,A
merchant_category_id,80,367,80,560,80
merchant_id,M_ID_e020e9b302,M_ID_86ec983688,M_ID_979ed661fc,M_ID_e6d5ae8ea6,M_ID_e020e9b302
month_lag,-8,-7,-6,-5,-11
purchase_amount,-0.703331,-0.733128,-0.720386,-0.735352,-0.722865


In [17]:
def aggregate_by_card_id(df):
    unique_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 
                   'subsector_id', 'purchase_Year', 'purchase_Month', 'purchase_Week', 'purchase_Day']
    aggs = {}
    for c in unique_cols:
        aggs[c] = ['nunique'] 
#     aggs['authorized_flag'] = ['mean', 'sum']
#     aggs['purchase_amount'] = ['sum', 'max', 'min', 'mean', 'std', 'var']
#     aggs['installments'] = ['sum', 'max', 'mean', 'std', 'var']
#     aggs['month_lag'] = ['sum', 'max', 'min', 'mean', 'std', 'var']
#     aggs['purchase_date'] = [np.min, np.max, np.ptp]
#     aggs['card_id'] = ['size']
#     aggs['purchased_on_weekend'] = ['sum', 'mean']
#     aggs['month_diff'] = ['mean', 'std', 'var']
    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum','max','min','mean','var']
    aggs['purchase_date'] = ['max','min', np.ptp]
    aggs['month_lag'] = ['max','min','mean','var']
    aggs['month_diff'] = ['mean', 'std', 'var']
    aggs['authorized_flag'] = ['sum', 'mean']
    aggs['purchased_on_weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']
    for col in ['category_2','category_3']:
        df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
        df[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum')
        aggs[col+'_mean'] = ['mean']
        aggs[col+'_sum'] = ['sum']

    new_df = df.groupby(['card_id']).agg(aggs)
    new_df.columns = ['_'.join(col).strip() for col in new_df.columns.values]
    new_df.reset_index(inplace=True)
    other_df = (df.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    new_df = pd.merge(other_df, new_df, on='card_id', how='left')
    new_df["purchase_date_ptp"] = new_df["purchase_date_ptp"].dt.days
    new_df['purchase_date_diff'] = (new_df['purchase_date_max'] - new_df['purchase_date_min']).dt.days
    new_df['purchase_date_average'] = new_df['purchase_date_diff']/new_df['card_id_size']
    new_df['purchase_date_uptonow'] = (datetime.datetime.today() - new_df['purchase_date_max']).dt.days
    return new_df

In [31]:
# dfs_e = [hist_trans_c_e, new_hist_trans_c_e]

In [18]:
%time hist_trans_agg, new_hist_trans_agg = [aggregate_by_card_id(df) for df in dfs]

CPU times: user 3min 9s, sys: 18.7 s, total: 3min 28s
Wall time: 2min 24s


### Add exta interpreted columns on aggregates

In [56]:
hist_trans.loc[hist_trans['authorized_flag'] == 0]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,purchase_Is_quarter_start,purchase_Is_year_end,purchase_Is_year_start,purchase_Elapsed,purchased_on_weekend,month_diff,category_2_mean,category_2_sum,category_3_mean,category_3_sum
115,0,C_ID_4e6213e9bc,88,0,0,A,842,M_ID_22c9cfa265,-10,-0.730379,...,False,False,False,1491569889,0,11,0.074568,1.329550e+06,0.358458,5.588325e+06
132,0,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-5,-0.723782,...,False,False,False,1505688027,1,11,0.074568,1.329550e+06,0.358458,5.588325e+06
148,0,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-5,-0.723782,...,False,False,False,1505688026,1,11,0.074568,1.329550e+06,0.358458,5.588325e+06
168,0,C_ID_4e6213e9bc,333,0,0,A,605,M_ID_c2ae34c2ef,0,-0.664262,...,False,False,False,1519124270,0,11,0.074568,1.329550e+06,0.358458,5.588325e+06
213,0,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-7,-0.738132,...,False,False,False,1499499211,1,11,0.074568,1.329550e+06,0.358458,5.588325e+06
294,0,C_ID_4e6213e9bc,88,0,0,A,278,M_ID_21e1552dab,-11,-0.734135,...,False,False,False,1490197988,0,11,0.074568,1.329550e+06,0.358458,5.588325e+06
306,0,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-7,-0.738132,...,False,False,False,1499499150,1,11,0.074568,1.329550e+06,0.358458,5.588325e+06
365,0,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-7,-0.738132,...,False,False,False,1499499169,1,11,0.074568,1.329550e+06,0.358458,5.588325e+06
368,0,C_ID_4e6213e9bc,88,0,0,A,278,M_ID_21e1552dab,-11,-0.734135,...,False,False,False,1490197972,0,11,0.074568,1.329550e+06,0.358458,5.588325e+06
400,0,C_ID_5037ff576e,322,0,1,B,278,M_ID_b61c7d1be0,-3,-0.595260,...,False,False,False,1504817839,0,13,0.180375,7.055912e+05,-0.404556,-4.724214e+06


In [58]:
temp = hist_trans.loc[hist_trans['card_id'] == 'C_ID_05042ebd55']

In [59]:
temp

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,purchase_Is_quarter_start,purchase_Is_year_end,purchase_Is_year_start,purchase_Elapsed,purchased_on_weekend,month_diff,category_2_mean,category_2_sum,category_3_mean,category_3_sum
29112043,0,C_ID_05042ebd55,-1,0,0,A,437,M_ID_ce1e7a7a85,-4,-0.643976,...,False,False,False,1509203806,1,10,0.074568,1.329550e+06,0.358458,5.588325e+06
29112044,1,C_ID_05042ebd55,20,0,0,A,422,M_ID_0de7fd2bef,-1,-0.692948,...,False,False,False,1515964968,1,11,0.180375,7.055912e+05,0.358458,5.588325e+06
29112045,1,C_ID_05042ebd55,-1,0,0,A,703,M_ID_051eacb2a3,-2,-0.716855,...,False,True,False,1514737655,1,10,0.074568,1.329550e+06,0.358458,5.588325e+06
29112046,1,C_ID_05042ebd55,-1,0,0,A,80,M_ID_9ecff0beda,0,-0.705585,...,False,False,False,1518006846,0,11,0.074568,1.329550e+06,0.358458,5.588325e+06
29112047,0,C_ID_05042ebd55,-1,0,0,A,80,M_ID_9ecff0beda,0,-0.705134,...,False,False,False,1518006816,0,11,0.074568,1.329550e+06,0.358458,5.588325e+06
29112048,1,C_ID_05042ebd55,-1,0,0,A,278,M_ID_814793f74b,0,-0.725871,...,False,False,False,1518188681,0,11,0.074568,1.329550e+06,0.358458,5.588325e+06


In [ ]:
temp[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum')

In [21]:
new_hist_trans_agg.head().T

,0,1,2,3,4
card_id,C_ID_00007093c1,C_ID_0001238066,C_ID_0001506ef0,C_ID_0001793786,C_ID_000183fdda
transactions_count,2,26,2,31,11
city_id_nunique,2,8,1,7,2
merchant_category_id_nunique,2,15,2,21,9
merchant_id_nunique,2,25,2,31,11
state_id_nunique,2,4,1,5,2
subsector_id_nunique,2,9,2,14,6
purchase_Year_nunique,1,1,1,1,1
purchase_Month_nunique,1,2,1,2,2
purchase_Week_nunique,2,9,2,6,7


In [39]:
def add_extra_cols_on_agg(df):
    df['inverse_avg_transactions_per_day'] = df['purchase_date_diff']/df['card_id_size']
    df['days_since_last_transaction'] = (datetime.datetime.today() - df['purchase_date_max']).dt.days
    df['repurchase_merchant_rate'] = df['transactions_count']/df['merchant_id_nunique']
    df['merchant_category_repurchase'] = df['merchant_category_id_nunique']/df['merchant_id_nunique']
    df['category_2_sum_sum'] = df['category_2_sum_sum']/df['transactions_count']
    df['category_3_sum_sum'] = df['category_3_sum_sum']/df['transactions_count']
    df['avg_spend_per_merchant'] = df['purchase_amount_sum']/df['merchant_id_nunique']
    df['avg_trans_per_merchant'] = df['transactions_count']/df['merchant_id_nunique']
    df['avg_spend_per_transaction'] = df['purchase_amount_sum']/df['transactions_count']
    return df

In [40]:
[hist_trans_agg, new_hist_trans_agg] = [add_extra_cols_on_agg(df) for df in [hist_trans_agg, 
                                                                             new_hist_trans_agg]]

In [41]:
hist_trans_agg.to_feather('hist_trans_agg')
new_hist_trans_agg.to_feather('new_hist_trans_agg')

In [42]:
hist_trans_agg = feather.read_dataframe('hist_trans_agg')
new_hist_trans_agg = feather.read_dataframe('new_hist_trans_agg')

### Load test & train DFs

In [29]:
PATH = 'data/elo/'

In [30]:
train, test = [pd.read_csv(f'{PATH}{c}') for c in ['train.csv', 'test.csv']]

### Add date part to test & train dfs

In [31]:
add_datepart(train, 'first_active_month', drop=False)
add_datepart(test, 'first_active_month', drop=False)

### Merge train & test with new & old transactions history

In [43]:
def join_dfs(left, right, left_on, right_on=None, suffix='_old'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=("", suffix))

In [44]:
train_df = join_dfs(train, new_hist_trans_agg, left_on='card_id')

In [45]:
train_df = join_dfs(train_df, hist_trans_agg, left_on='card_id', suffix='_old')

In [46]:
test_df = join_dfs(test, new_hist_trans_agg, left_on='card_id')
test_df = join_dfs(test_df, hist_trans_agg, left_on='card_id', suffix='_old')

In [47]:
test_df.shape, train_df.shape

((123623, 120), (201917, 121))

### Add extra columns like age, total transactions, installments, purchase amount, first buy etc

In [48]:
for df in [train_df, test_df]:
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['card_id_total'] = df['card_id_size']+df['card_id_size_old']
    df['purchase_amount_total'] = df['purchase_amount_sum']+df['purchase_amount_sum_old']
    df['installments_total'] = df['installments_sum'] + df['installments_sum_old']
    df['hist_first_buy'] = (df['purchase_date_min_old'] - df['first_active_month']).dt.days
    df['new_first_buy'] = (df['purchase_date_min'] - df['first_active_month']).dt.days
    df['avg_spend_per_transaction'] = df['purchase_amount_total']/df['card_id_total']

### Mark the outliers

In [49]:
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -30, 'outliers'] = 1
train_df['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

### Target encode the outliers 

In [51]:
train_df.shape, test_df.shape

((201917, 128), (123623, 126))

In [52]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = train_df.groupby([f])['outliers'].mean()
    train_df[f] = train_df[f].map(order_label)
    test_df[f] = test_df[f].map(order_label)

In [53]:
train_df.to_feather('train_df')
test_df.to_feather('test_df')